In [ ]:
import pandas as pd
import requests 
import json 
import csv
import time
import datetime

In [ ]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [ ]:
def collectSubData(subm):
    subData = {} #list to store data points
    subData['title'] = subm['title']
    subData['url'] = subm['url']
    try:
        subData['flair'] = subm['link_flair_text']
    except KeyError:
        subData['flair'] = "NaN"    
    subData['author'] = subm['author']
    subData['sub_id'] = subm['id']
    subData['score'] = subm['score']
    subData['created'] = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    subData['numComms'] = subm['num_comments']
    subData['permalink'] = subm['permalink']
    subStats[subm['id']] = subData

In [ ]:
after = int(time.mktime(datetime.date(2020,1,6).timetuple()))
before = int(time.mktime(datetime.date(2021,4,28).timetuple()))
query = "exxon" 
sub = "personalfinance"
subCount = 0
subStats = {}

In [ ]:
data = getPushshiftData(query, after, before, sub)
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
print(str(len(subStats)) + " submissions have added to list")

In [ ]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 25)

df_subStats = pd.DataFrame.from_dict(subStats.values())
df_subStats.groupby(by='created').sum()